In [10]:
%%capture
from __future__ import print_function, division
import appmode
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import Output
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import Button, Layout
from IPython.display import Javascript
import pandas as pd
from IPython.display import display as display
from IPython.display import HTML
from ipywidgets import HBox, VBox
from collections import OrderedDict
from fuzzywuzzy import process 
from fuzzywuzzy import fuzz
pd.options.display.html.table_schema = True
pd.options.display.max_rows = None
pd.options.display.max_columns = 10
pd.options.display.max_colwidth = 100
pd.options.display.width = 300
pd.options.display.expand_frame_repr = True
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from io import StringIO
import csv
import os
import os.path
import re
from bs4 import BeautifulSoup as BeautifulSoup
from openpyxl import Workbook as Workbook
from diff_match_patch import diff_match_patch
#import pixiedust
#import pixiedust_node
#import pixiedust_rosie



<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#The-H&amp;K-Lawyer-Notebook" data-toc-modified-id="The-H&amp;K-Lawyer-Notebook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>The H&amp;K Lawyer Notebook</a></span><ul class="toc-item"><li><span><a href="#Classification-Tool" data-toc-modified-id="Classification-Tool-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Classification Tool</a></span></li><li><span><a href="#Data-Tool" data-toc-modified-id="Data-Tool-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data Tool</a></span></li><li><span><a href="#Bulk-Compare" data-toc-modified-id="Bulk-Compare-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Bulk Compare</a></span></li><li><span><a href="#Fuzzy-Searching" data-toc-modified-id="Fuzzy-Searching-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Fuzzy Searching</a></span></li><li><span><a href="#Find-Precedent" data-toc-modified-id="Find-Precedent-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Find Precedent</a></span></li><li><span><a href="#QuickCAN-(Coming-Soon)" data-toc-modified-id="QuickCAN-(Coming-Soon)-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>QuickCAN <em>(Coming Soon)</em></a></span></li><li><span><a href="#QuickCAR-(Coming-Soon)" data-toc-modified-id="QuickCAR-(Coming-Soon)-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>QuickCAR <em>(Coming Soon)</em></a></span></li><li><span><a href="#Signature-Page-Generator" data-toc-modified-id="Signature-Page-Generator-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Signature Page Generator</a></span></li><li><span><a href="#Closing-Binder-Generator" data-toc-modified-id="Closing-Binder-Generator-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Closing Binder Generator</a></span></li></ul></li></ul></div>

In [11]:
def save_list_as_excel(path, clean_sent_list, raw_sent_list):
    wb = Workbook()
    ws = wb.active
    column_cell_A = 'A'
    column_cell_B = 'B'
    column_cell_C = 'C'
    ws[column_cell_A + str(1)] = 'text'
    ws[column_cell_B + str(1)] = 'label'
    ws[column_cell_C + str(1)] = 'raw'
    list_len = len(clean_sent_list)
    for i in range(0, list_len):
        ws[column_cell_A + str(i + 2)] = clean_sent_list[i]
        ws[column_cell_B + str(i + 2)] = 'label'
        ws[column_cell_C + str(i + 2)] = raw_sent_list[i]
    wb.save(path)
    print('Excel write complete')


def html_from_file_no_tags(file_path):
    with open(file_path, 'rb') as myfile:
        raw_text = myfile.read()
    bsObj = BeautifulSoup(raw_text, 'lxml').text
    return bsObj


def normalize_sent(sent):
    wpt = nltk.WordPunctTokenizer()
    stop_words = nltk.corpus.stopwords.words('english')
    sent = re.sub(r'[^a-zA-Z\s]', '', sent, re.I | re.A)
    sent = sent.strip()
    tokens = wpt.tokenize(sent)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    sent = ' '.join(filtered_tokens).lower()
    return sent


def html_from_file_no_tags(file_path):
    with open(file_path, 'rb') as myfile:
        raw_text = myfile.read()
    bsObj = BeautifulSoup(raw_text, 'lxml').text
    return bsObj


def load_normal_with_stopwords(path):
    doc = str(load_raw(path))
    return normalize_document_return_list(doc)


def load_normal_no_stopwords(path):
    doc = load_raw(path)
    norm = normalize_document_return_list(doc)
    clean_sent = []
    for sent in norm:
        clean = remove_stop_words(sent)
        clean_sent.append(clean)
    return clean_sent


def list_from_directory(path):
    list_of_text = []
    for file in os.listdir(path):
        filename = os.fsdecode(file)
        file_path = path + filename
        text = load_raw(file_path)
        text = str(text)
        list_of_text.append(text)


def load_excel(path):
    print('finish')


def load_list_from_csv(path):
    with open(path, 'r') as f:
        reader = csv.reader(f)
        list_raw = list(reader)
        list_clean = []
        for i in list_raw:
            x = ''.join(i)
            list_clean.append(x)
        list_clean_two = []
        for i in list_clean:
            x = ''.join(i)
            list_clean_two.append(x)
        return list_clean_two


def load_raw(path, tags=False):
    if path.endswith('html'):
        if (tags):
            return html_from_file_tags(path)
        else:
            return html_from_file_no_tags(path)
    elif path.endswith('.txt'):
        return str(text_from_file(path))
    else:
        try:
            return str(text_from_binary(path))
        except:
            print('Failed to load as binary. Try reader that accepts url as argument (e.g., html_from_web_tags(url) or html_from_web_no_tags(url)).')


def text_from_binary(file_path):
    #text = textract.process(file_path, method='tesseract', language='eng')
    text = 'Not set up for binary'
    return text.decode('unicode_escape').encode('utf-8', 'ignore').strip()


def html_from_file_tags(file_path):
    with open(file_path, 'rb') as myfile:
        raw_text = myfile.read()
    return raw_text


def html_from_web_no_tags(url):
    response = urlopen(url)
    bsObj = BeautifulSoup(response, 'lxml').text
    return bsObj


def html_from_web_tags(url):
    response = urlopen(url)
    tagged_text = response.read()
    return tagged_text


def text_from_file(file_path):
    with open(file_path, 'rb') as myfile:
        raw_text = myfile.read()
    return raw_text


def remove_stop_words(doc):
    wpt = nltk.WordPunctTokenizer()
    stop_words = nltk.corpus.stopwords.words('english')
    tokens = wpt.tokenize(doc)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    doc = ' '.join(filtered_tokens).lower()
    return doc


def normalize_document_return_list(doc):
    wpt = nltk.WordPunctTokenizer()
    stop_words = nltk.corpus.stopwords.words('english')
    list_of_clean_sents = []
    sent_list = tokenize.sent_tokenize(str(doc))
    for sent in sent_list:
        sent = re.sub(r'[^a-zA-Z\s]', '', sent, re.I | re.A)
        sent = sent.strip()
        tokens = wpt.tokenize(sent)
        filtered_tokens = [token for token in tokens if token not in stop_words]
        sent = ' '.join(filtered_tokens).lower()
        list_of_clean_sents.append(sent)
    return list_of_clean_sents


def tokenize_and_stem(text):
    stemmer = SnowballStemmer("english")
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    joined_text = " ".join(stems)
    print(joined_text)
    return joined_text


def create_progress_bar():
    in_progress = widgets.IntProgress(
    value=0,
    min=0,
    max=10,
    step=1,
    visible=False,
    description='Status:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)
    return in_progress

def change_progress_bar(name, percent_complete):
    name.value = percent_complete

def is_progress_bar_visible(name, visible):
    name.visible = visible
    
def clear_output(name):
    name.close
    
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

def plot3D():
    X = np.arange(-5, 5, 0.25)
    Y = np.arange(-5, 5, 0.25)
    X, Y = np.meshgrid(X, Y)
    R = np.sqrt(X**2 + Y**2)
    Z = np.sin(R)
    # Make the plot
    fig = plt.figure()
    ax = fig.gca(projection="3d")
    surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
    ax.set_zlim(-1.01, 1.01)
    fig.colorbar(surf, shrink=0.5, aspect=5)
    plt.show()
    
def create_submit_button():
    button = widgets.Button(
    description='Submit',
    disabled=False
    )
    button.style.button_color = 'steelblue'
    button.layout={'border': '1px solid grey'}
    return button

def create_download_button():
    button = widgets.Button(
    description='Download Excel',
    disabled=False
    )
    button.style.button_color = 'Honeydew'
    button.layout={'border': '1px solid grey'}
    return button

def create_refresh_button():
    button = widgets.Button(
    description='Clear Output',
    disabled=False
    )
    button.style.button_color = 'LightSlateGray'
    button.layout={'border': '1px solid grey'}
    return button

def create_chart_button():
    button = widgets.Button(
    description='Chart_It',
    disabled=False
    )
    button.style.button_color = 'LightSkyBlue'
    button.layout={'border': '1px solid grey'}
    return button


def CleanAndSplitText(frame):
    textDataOut = [] 
    reHeaders = re.compile(r" *TABLE OF CONTENTS:? *"
                           "| *Title [IVXLC]+:? *"
                           "| *Section [IVXLC]+:? *"
                           "| *Article [IVXLC]+:? *"
                           "| *Subtitle [A-Z]+:? *"
                           "| *\(Sec\. \d+\) *")
    rePhraseBreaks = re.compile("[\"\!\?\)\]\}\,\:\;\*\-]*\s+\([0-9]+\)\s+[\(\[\{\"\*\-]*"                             
                                "|[\"\!\?\)\]\}\,\:\;\*\-]+\s+[\(\[\{\"\*\-]*"
                                "|\.\.+"
                                "|\s*\-\-+\s*"
                                "|\s+\-\s+"
                                "|\:\:+"
                                "|\s+[\/\(\[\{\"\-\*]+\s*"
                                "|[\,!\?\"\)\(\]\[\}\{\:\;\*](?=[a-zA-Z])"
                                "|[\"\!\?\)\]\}\,\:\;]+[\.]*$"
                             )
    regexUnderbar = re.compile('_')
    regexSpace = re.compile(' +')
    regexPeriod = re.compile("\.$")
    for i in range(0, len(frame)):     
        docID = frame['ID'][i]
        docText = str(frame['Text'][i])
        lineIndex=0;
        sections = reHeaders.split(docText)
        for section in sections:
            sentences = tokenize.sent_tokenize(section)
            for sentence in sentences:
                textSegs = rePhraseBreaks.split(sentence)
                numSegs = len(textSegs)
                for j in range(0,numSegs):
                    if len(textSegs[j])>0:               
                        textSegs[j] = regexUnderbar.sub(" ",textSegs[j])
                        words = regexSpace.split(textSegs[j])
                        phraseOut = ""
                        words[-1] = regexPeriod.sub("", words[-1])
                        if "\." in words[-1]:
                            words[-1] += "."
                        phraseOut = " ".join(words)  
                        textDataOut.append([docID, lineIndex, phraseOut])
                        lineIndex += 1
    frameOut = pandas.DataFrame(textDataOut, columns=['DocID', 'DocLine', 'CleanedText'])                      
    return frameOut

![alt text](https://s3.amazonaws.com/blaze4/HK+logo.png)

# The H&K Lawyer Notebook

A collection of simple, yet powerful tools for lawyers to do their jobs better and more efficiently. By leveraging data analytics, natural language processing, machine learning, and other technology solutions, users are able to produce better quality work faster and e more efficient. Many of these tools are still under active development, so the H&K Lawyer Notebook is reserved for our more intrepid lawyers.

***

## Classification Tool

Classify text using one of several trained machine learning models.

In [12]:
#TODO Create seperate lists for individual items for other subject matters
#TODO Add training back-end

results = ''
out = widgets.Output(layout={'border': '.25px solid grey'})
out_aux = widgets.Output()

model_selection = widgets.RadioButtons(
    options=['Google AutoML', 'SKLearn', 'TensorFlow', 'Keras', 'Gensim'],
     value='SKLearn',
    description='Platform:',
    disabled=False
)

subject_matter_selection = widgets.RadioButtons(
    options=['Litigation', 'Finance', 'Leasing', 'Private Equity', 'Corporate M&A', 'Bond Financing'],
     value='Finance',
    description='Practice:',
    disabled=False
)

file_path_text = widgets.Text(              
    value='credit.txt',
    placeholder='filename',
    description='Filename:',
    disabled=False
)

#if subject_matter_selection == 'Finance':
clause_selection = widgets.SelectMultiple(
    options=['All', 'MAC_definition', 'commitment_fee', 'change_control', 'change_control_prepayment', 'debt_issuance_prepayment', 'default_interest', 'disposition_assets_prepayment', 'equity_issuance_prepayment', 'eurodollar_rate', 'excess_cash_flow', 'fixed_charge_ratio', 'lender_inspection_rights', 'margin_leverage_ratio', 'margin_rating', 'monetary_default', 'net_worth', 'restricted_payments', 'Amendments_Consent', 'secured_facility', 'Amendments_Consent', 'unused_fee', 'voluntary_prepayment'],
    value=['All'],
    rows=10,
    description='Clause selection:',
    disabled=False
)


model_name = widgets.RadioButtons(
    options=['SVM', 'LSTM', 'NB'],
    value='SVM',
    description='Model:',
    disabled=False
)

save_file_path = widgets.Text(              
    value='model.pickle',
    placeholder='filename',
    description='Path to Save:',
    disabled=False
)

model_training_material = widgets.Text(              
    value='training.xlsx',
    placeholder='file path',
    description='Train Docs:',
    disabled=False
)

button = create_submit_button()
download_button = create_download_button()
refresh_button = create_refresh_button()
tab1 = VBox(children=[HBox(children=[file_path_text])])
tab2 = VBox(children=[HBox(children=[model_selection, subject_matter_selection, clause_selection])])
tab3 = VBox(children=[HBox(children=[model_name, model_training_material, save_file_path])])
tab = widgets.Tab(children=[tab2, tab1, tab3])
tab.set_title(0, 'Configure')
tab.set_title(1, 'Predict')
tab.set_title(2, 'Training')
HBox_classify = HBox(children=[button, refresh_button, download_button])
VBox_classify = VBox(children=[tab, HBox_classify])
VBox_classify.layout={'border': '.25px solid grey'}

with out_aux:
    display(VBox_classify)

returned_values = {} 

def clear_output():
    out.clear_output()
    

@download_button.on_click
def download_button_clicked(b):
    output = returned_values['df_output']
    writer = pd.ExcelWriter('classification_results.xlsx')
    output.to_excel(writer,'Sheet1')
    writer.save()
    download_button.description = 'Download complete'

@refresh_button.on_click
def refresh_button_clicked(b):
    out.clear_output()
    download_button.description = 'Download Excel'

#@out.capture()
@button.on_click
def on_button_clicked(b):
    clear_output()
    download_button.description = 'Download Excel'
    in_progress = create_progress_bar()
    display(in_progress)
    change_progress_bar(in_progress, 0)
    #Load training corpus
    #df = pd.read_excel('/home/ec2-user/SageMaker/data_b.xlsx') #Uncomment if running SageMaker
    df = pd.read_excel('https://s3.amazonaws.com/blaze4/data_b.xlsx') #Uncomment if running local
    df = df.sample(frac=1, axis=1).reset_index(drop=True)
    df = df[pd.notnull(df['Text'])]
    df = df.drop_duplicates()
    change_progress_bar(in_progress, 2)
    col = ['Label', 'Text']
    df = df[col]
    df.columns = ['Label', 'Text']
    df = df.sample(frac=1).reset_index(drop=True)
    #Vectorize and transform training corpus
    df['category_id'] = df['Label'].factorize()[0]
    change_progress_bar(in_progress, 4)
    category_id_df = df[['Label', 'category_id']].drop_duplicates().sort_values('category_id')
    category_to_id = dict(category_id_df.values)
    id_to_category = dict(category_id_df[['category_id', 'Label']].values)
    #Train model
    X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Label'], random_state = 0)
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(X_train)
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    clf = LinearSVC().fit(X_train_tfidf, y_train)
    change_progress_bar(in_progress, 6)
    #Load document to be reviewed, clause segment and normalize
    current_raw_sent_list = []
    clean_raw_list = []
    current_clean_sent_list = []
    sent_list = sent_tokenize(load_raw(file_path_text.value))
    for sent in sent_list:
        clean_sent = normalize_sent(sent)
        current_clean_sent_list.append(clean_sent)
        current_raw_sent_list.append(sent)
    in_progress.value = 7
    change_progress_bar(in_progress, 7)
    for sent in current_raw_sent_list:
        clean_raw = re.sub("\xe2", " ", sent)
        clean_raw = re.sub("\x80", " ", clean_raw)
        clean_raw = re.sub(r"\n\n", " ", clean_raw)
        clean_raw = re.sub(r'\n', " ", clean_raw)
        clean_raw = re.sub("\xe2\x80\x9c", " ", clean_raw)
        clean_raw = re.sub("each\nRevolving+", "each Revolving", clean_raw)
        clean_raw = re.sub("\xe2\x80\x9cRequired", "Required", clean_raw)
        clean_raw = re.sub("\xe2\x80\x9c", ' ', clean_raw)
        clean_raw = re.sub("\xe2\x80\x99s", ' ', clean_raw)
        clean_raw = re.sub("\sn\s", ' ', clean_raw)
        clean_raw = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", clean_raw)
        clean_raw_list.append(clean_raw)
    #Load normalized sentences into dataframe
    df_current = pd.DataFrame(columns=['label', 'raw', 'clean'])
    df_current['clean'] = current_clean_sent_list
    df_current['raw'] = clean_raw_list
    change_progress_bar(in_progress, 8)
    #Iterate through dataframe, predict clause type and record to [label] column in df
    for index, row in df_current.iterrows():
        test = []
        test.append(df_current['clean'][index])
        X_test=count_vect.transform(test)
        z = clf.predict(X_test)
        df_current['label'][index] = z[0]
    change_progress_bar(in_progress, 10)
    if clause_selection.value[0] == 'All':
        clause_selection.value = ['MAC_definition', 'commitment_fee', 'change_control', 'change_control_prepayment', 'debt_issuance_prepayment', 'default_interest', 'disposition_assets_prepayment', 'equity_issuance_prepayment', 'eurodollar_rate', 'excess_cash_flow', 'fixed_charge_ratio', 'lender_inspection_rights', 'margin_leverage_ratio', 'margin_rating', 'monetary_default', 'net_worth', 'restricted_payments', 'Amendments_Consent', 'secured_facility', 'Amendments_Consent', 'unused_fee', 'voluntary_prepayment']
    df_output = df_current[df_current.apply(lambda x: x['label'] in clause_selection.value, axis=1)]
    returned_values['df_output'] = df_output 
    in_progress.close()                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
    with out:
        display(df_output)
display(out_aux)
out


Output()

Output(layout=Layout(border='.25px solid grey'))

## Data Tool

A tool for organizing and visualizing data.

In [13]:
#TODO design user input box; add charting; replace data set with car ride specific examples.

In [14]:
data_out = widgets.Output(layout={'border': '1px solid grey'})
data_UI_out = widgets.Output()


chart_submit_button = create_chart_button()
data_button = create_submit_button()
data_download_button = create_download_button()
data_refresh_button = create_refresh_button()

@data_download_button.on_click
def data_download_button_clicked(b):
    output = returned_values['df_min']
    writer = pd.ExcelWriter('data_results.xlsx')
    output.to_excel(writer,'Sheet1')
    writer.save()
    data_download_button.description = 'Download complete'

def clear_output():
    data_out.clear_output()
    
@data_refresh_button.on_click
def data_refresh_button_clicked(b):
    clear_output()

@chart_submit_button.on_click
#@data_out.capture(clear_output=True, wait=True)
def chart_it(b):
    c_in_progress = create_progress_bar()
    display(c_in_progress)
    change_progress_bar(c_in_progress, 2)
    if data_selection.value == 'employee_set':
        df = pd.read_csv('https://s3.amazonaws.com/blaze4/HR_comma_sep.csv')
    else:
        df = pd.read_csv('https://s3.amazonaws.com/blaze4/trip_stats_taz.csv')
        #df = pd.read_csv('/Users/josiasdewey/jupyter/notebooks/static/trip_stats_taz.csv')
    df_min = df[0:2000]
    change_progress_bar(c_in_progress, 4)
    if chart_selection.value == '3D Plot':
        plot3D()
        change_progress_bar(c_in_progress, 10)
        c_in_progress.close()
    else:
        bp_x = df_min[data_points_selection_x.value]
        bp_y = df_min[data_points_selection_y.value]
        bp_z = df_min[data_points_selection_z.value]
        if chart_selection.value == '3D Plot':
            plot3D()
        if chart_selection.value == 'Scatter Plot':
            cht = plt.scatter(bp_x, bp_y,
                 color="blue", label=r"Scatter")
        if chart_selection.value == 'Bar Graph':
            cht = plt.bar(bp_x, bp_y,
                 color="blue", label=r"Bar")
        if chart_selection.value == 'Bar Graph':
            cht = plt.bar(bp_x, bp_y,
                 color="blue", label=r"Bar")
        xlim_max = (1.25 * (df_min[data_points_selection_x.value].max()))
        ylim_max = (1.25 * (df_min[data_points_selection_y.value].max()))
        xlim_min = (-1.25 * (df_min[data_points_selection_x.value].min()))
        ylim_min = (-1.25 * (df_min[data_points_selection_y.value].min()))
        change_progress_bar(c_in_progress, 6)
        plt.xlim(xlim_min, xlim_max)
        plt.ylim(ylim_min, ylim_max)
        plt.xlabel(data_points_selection_x.value)
        plt.ylabel(data_points_selection_y.value)
        plt.title(r"Relationship between %s and %s" % (data_points_selection_x.value, data_points_selection_y.value) )
        fig_size = plt.rcParams["figure.figsize"]
        fig_size[0] = 18
        fig_size[1] = 9
        plt.rcParams["figure.figsize"] = fig_size
        plt.legend(loc="lower left")
        change_progress_bar(c_in_progress, 8)
        plt.ion()
        plt.show()
        change_progress_bar(c_in_progress, 10)
        c_in_progress.close()
        with data_out:
            display(cht)
    
#@data_out.capture(clear_output=True, wait=True)
@data_button.on_click
def load_data(b):
    clear_output()
    d_in_progress = create_progress_bar()
    display(d_in_progress)
    change_progress_bar(d_in_progress, 2)
    if data_selection.value == 'employee_set':
        df = pd.read_csv('https://s3.amazonaws.com/blaze4/HR_comma_sep.csv')
    else:
        df = pd.read_csv('https://s3.amazonaws.com/blaze4/trip_stats_taz.csv')
        #df = pd.read_csv('/Users/josiasdewey/jupyter/notebooks/static/trip_stats_taz.csv')
    df_min = df[0:2000]
    change_progress_bar(d_in_progress, 4)
    returned_values['df_data'] = df_min
    change_progress_bar(d_in_progress, 8)
    data_points_selection_x.options = list(df_min)
    data_points_selection_y.options = list(df_min)
    data_points_selection_z.options = list(df_min)
    change_progress_bar(d_in_progress, 10)
    d_in_progress.close()
    returned_values['df_min'] = df_min
    with data_out:
        display(df_min)
    
custom_file_path_text = widgets.Text(              
    value='',
    placeholder='file path',
    description='Filename:',
    disabled=False
)

data_selection = widgets.SelectMultiple(
    options=['employee_set', 'ride_share_data_SF'],
    value=['employee_set'],
    rows=10,
    description='Data Set:',
    disabled=False
)

chart_selection = widgets.RadioButtons(
    options=['Scatter Plot', '3D Plot', 'Bar Graph', 'Pie Graph', 'Line Graph'],
    value='Scatter Plot',
    description='Select chart:',
    disabled=False
)

returned_values = {'df_data': ['load data', 'load data']}
df = returned_values['df_data']
df_col = list(df)
data_points_selection_x = widgets.RadioButtons(
    options=df_col,
    value='load data',
    description='x-axis:',
    disabled=False
)

data_points_selection_y = widgets.RadioButtons(
    options=df_col,
    value='load data',
    description='y-axis:',
    disabled=False
)

data_points_selection_z = widgets.RadioButtons(
    options=df_col,
    value='load data',
    description='z-axis:',
    disabled=False
)

tab1 = VBox(children=[HBox(children=[data_selection, custom_file_path_text])])
tab2 = VBox(children=[HBox(children=[chart_selection, 
                                     data_points_selection_x,
                                     data_points_selection_y,
                                     data_points_selection_z])])
tab = widgets.Tab(children=[tab1, tab2])
tab.set_title(0, 'Data Sets')
tab.set_title(1, 'Charting')
HBox_data = HBox(children=[data_button, data_refresh_button, chart_submit_button, data_download_button])
VBox_data = VBox(children=[tab, HBox_data])
VBox_data.layout={'border': '.25px solid grey'}
with data_UI_out:
    display(VBox_data)
    
display(data_UI_out)
data_out

Output()

Output(layout=Layout(border='1px solid grey'))

## Bulk Compare

Use this bulk comparison tool to compare a large number of documents against a common base form.  For example, compare lease agreements against a lease form to determine deviations from approved form. 

In [15]:
compare_out = widgets.Output(layout={'border': '.25px solid grey'}
)
compare_UI_out = widgets.Output()
compare_button = create_submit_button()
compare_refresh_button  = create_refresh_button()
compare_download_button  = create_download_button()

@compare_button.on_click
def on_click(b):
    doc_1_raw_sent_list = []
    doc_1_clean_sent_list = []
    base_raw = text_from_file(compare_file_path_text_1.value)
    sent_list = sent_tokenize(str(base_raw))
    for sent in sent_list:
        clean_sent = normalize_sent(sent)
        doc_1_clean_sent_list.append(clean_sent)
        doc_1_raw_sent_list.append(sent)
    df_doc_1 = pd.DataFrame(columns=['raw', 'clean'])
    df_doc_1['clean'] = doc_1_clean_sent_list
    df_doc_1['raw'] = doc_1_raw_sent_list
    
    doc_2_raw_sent_list = []
    doc_2_clean_sent_list = []
    compare_raw = text_from_file(compare_file_path_text_2.value)
    sent_list = sent_tokenize(str(compare_raw))
    for sent in sent_list:
        clean_sent = normalize_sent(sent)
        doc_2_clean_sent_list.append(clean_sent)
        doc_2_raw_sent_list.append(sent)
    df_doc_2 = pd.DataFrame(columns=['raw', 'clean'])
    df_doc_2['clean'] = doc_2_clean_sent_list
    df_doc_2['raw'] = doc_2_raw_sent_list
    df_results = pd.DataFrame(columns=['query', 'match', 'strength', 'compare'])
    query = df_doc_1['clean']
    choices = df_doc_2['clean']
    query_list = []
    found_list = []
    strength_list = []
    for q in query:
        match = process.extractOne(q, choices=choices, scorer=fuzz.token_sort_ratio, score_cutoff=0)
        if match != None:
            index = match[2]
            found = match[0]
            strength = match[1]
            query_list.append(q)
            found_list.append(choices[index])
            strength_list.append(strength)
    df_results['query'] = query_list
    df_results['found']= found_list
    df_results['strength'] = strength_list
    #precedent = df_master["Result"] 
    #current = df_master["Query"] 
    compared_file = []
    compared_clause = [] 
    clean_clause = []
    dmp = diff_match_patch()
    dmp.Diff_Timeout = 0
    for index, row in df_results.iterrows():
        diff = dmp.diff_main(str(row['found']), str(row['query']))
        view = dmp.diff_prettyHtml(diff) 
        #display(HTML(view))
        compared_file.append(view)
        joined = ' '.join(compared_file)
    with open('test.html', 'w+') as f:
        f.write(joined)
    df_results['compare'] = compared_file
    with compare_out:
        #display(HTML(df_results.to_html()))
    #return df_results
        html_1 = """
<!DOCTYPE html>
<html>
<head>
<style>
table {
    font-family: arial, sans-serif;
    border-collapse: collapse;
    width: 100%;
}

td, th {
    border: 1px solid #dddddd;
    text-align: left;
    padding: 8px;
}

tr:nth-child(even) {
    background-color: #dddddd;
}
</style>
</head>
<body>

<h2>Comparison Table</h2>

<table>
  <tr>
    <th>Current</th>
    <th>Proposed</th>
    <th>Comparison</th>
  </tr>
    """
        html_rows = []
        rows =len (compared_file)
        for i in range(rows):
            html_row = "<tr>" + "<th>current</th>" + "<th>proposed</th>"+ "<th>" + str(compared_file[i]) + "</th></tr>"
            html_rows.append(html_row)
        html_rows.append('</table></body></html>')
    html_2 = ''.join(compared_file)
    html = html_1 + html_2
    display(HTML(html))

compare_file_path_text_1 = widgets.Text(
    value='Stock Purchase Agreement_original.txt',
    placeholder='file path',
    description='Base document:',
    disabled=False
)

compare_file_path_text_2 = widgets.Text(
    value='Stock Purchase Agreement_new_deal.txt',
    placeholder='',
    description='Docs to compare:',
    disabled=False
)

compare_output_path = widgets.Text(
    value='',
    placeholder='file path for output',
    description='output path:',
    disabled=False
)

#test comment

timeout_slider = widgets.FloatSlider(
    value=0,
    base=1,
    min=0, # max exponent of base
    max=1, # min exponent of base
    step=0.1, # exponent step
    description='Timeout:'
)


tab1 = VBox(children=[HBox(children=[compare_file_path_text_1, compare_file_path_text_2])])
tab2 = VBox(children=[HBox(children=[timeout_slider])])
tab = widgets.Tab(children=[tab1, tab2])
tab.set_title(0, 'Documents')
tab.set_title(1, 'Settings')
HBox_compare = HBox(children=[compare_button, compare_refresh_button, compare_download_button])
VBox_compare = VBox(children=[tab, HBox_compare])
VBox_compare.layout={'border': '.25px solid grey'}

with compare_UI_out:
    display(VBox_compare)
display(compare_UI_out)
compare_out

Output()

Output(layout=Layout(border='.25px solid grey'))

## Fuzzy Searching

Use "fuzzy" searching to compare any number of items (the 'Query') against a large groups of possible matches (the 'Search'). For example, a client requests we confirm the destruction of files relating to several hundred matters, where the matter description for each must be cross checked against several thousand entries in our file storage list. This tool leverages fuzzy matching in order to identify matches undetected by pure booleen searches (e.g., "FIRST INTERNATIONAL" captured even if search query is "INTL FIRS")

In [16]:
#TO DO -- Finish VBox layout

fuzzy_out = widgets.Output(layout={'border': '.25px solid grey'}
)
fuzzy_UI_out = widgets.Output()

fuzzy_button = create_submit_button()
fuzzy_download_button = create_download_button()
fuzzy_refresh_button = create_refresh_button()

@fuzzy_button.on_click
def on_click(b):
    doc_1_raw_sent_list = []
    doc_1_clean_sent_list = []
    doc_1_raw = load_raw(file_path_text_1.value)
    sent_list = sent_tokenize(str(doc_1_raw))
    for sent in sent_list:
        clean_sent = normalize_sent(sent)
        doc_1_clean_sent_list.append(clean_sent)
        doc_1_raw_sent_list.append(sent)
    df_doc_1 = pd.DataFrame(columns=['raw', 'clean'])
    df_doc_1['clean'] = doc_1_clean_sent_list
    df_doc_1['raw'] = doc_1_raw_sent_list
    doc_2_raw_sent_list = []
    doc_2_clean_sent_list = []
    doc2_raw = load_raw(file_path_text_2.value)
    sent_list = sent_tokenize(str(doc2_raw))
    for sent in sent_list:
        clean_sent = normalize_sent(sent)
        doc_2_clean_sent_list.append(clean_sent)
        doc_2_raw_sent_list.append(sent)
    df_doc_2 = pd.DataFrame(columns=['raw', 'clean'])
    df_doc_2['clean'] = doc_2_clean_sent_list
    df_doc_2['raw'] = doc_2_raw_sent_list
    df_results = pd.DataFrame(columns=['query', 'match', 'strength'])
    query = df_doc_1['clean']
    choices = df_doc_2['clean']
    query_list = []
    found_list = []
    strength_list = []
    for q in query:
        match = process.extractOne(q, choices=choices, scorer=fuzz.token_sort_ratio, score_cutoff=0)
        if match != None:
            index = match[2]
            if match[1] >= fuzzy_slider.value:
                found = match[0]
                strength = match[1]
            else:
                found = 'None'
                strength = 'N/A'
            query_list.append(q)
            found_list.append(choices[index])
            strength_list.append(strength)
    df_results['query'] = query_list
    df_results['found']= found_list
    df_results['strength'] = strength_list
    display(df_results)
    

file_path_text_1 = widgets.Text(
    value='query_set.xlsx',
    placeholder='Type something',
    description='Query:',
    disabled=False
)

file_path_text_2 = widgets.Text(
    value='search_set.xlsx',
    placeholder='Type something',
    description='Search:',
    disabled=False
)

number_returned = widgets.Dropdown(
    options=['1', '2', '3', '4'],
    value='2',
    description='Number returned:',
    disabled=False,
)

t = widgets.Text(
    value='filename and path',
    placeholder='Type something',
    description='Compare:',
    disabled=False
)

fuzzy_slider = widgets.FloatSlider(
    value=0,
    base=100,
    min=0, # max exponent of base
    max=100, # min exponent of base
    step=0.2, # exponent step
    description='Similarity:'
)

tab1 = VBox(children=[HBox(children=[file_path_text_1, file_path_text_2])])
tab2 = VBox(children=[HBox(children=[number_returned, fuzzy_slider])])
tab = widgets.Tab(children=[tab1, tab2])
tab.set_title(0, 'Query Parameters')
tab.set_title(1, 'Configure')
HBox_fuzzy = HBox(children=[fuzzy_button, fuzzy_refresh_button, fuzzy_download_button])
VBox_fuzzy = VBox(children=[tab, HBox_fuzzy])
VBox_fuzzy.layout={'border': '.25px solid grey'}

with fuzzy_UI_out:
    display(VBox_fuzzy)
    
display(fuzzy_UI_out)
fuzzy_out

Output()

Output(layout=Layout(border='.25px solid grey'))

## Find Precedent
Quickly search one or more curated clause banks for similar clauses.  Also, the user can identify form documents from which a particular agreememt originates.  The user can set a minimum similarity threshold to filter out less relevant clauses.  

In [17]:

precedent_out = widgets.Output(layout={'border': '.25px solid grey'})
precedent_UI_out = widgets.Output()

precedent_button = create_submit_button()
precedent_download_button = create_download_button()
precedent_refresh_button = create_refresh_button()

class Document():
    
    def __init__(self,id_num):
        self.id_num = id_num
        self.text = ''
        self.sections = []
        self.definitions = []
        
    def is_section_related(self, text):
        for section in self.sections:
            if section.text == text:
                return section.id_num
        return('This Document does not contain that section.')

    @property
    def document_type(self):
        pass
    
    @property
    def author(self):
        pass
    
    @property
    def favors(self):
        pass

class Section(Document):
    
    def __init__(self,id_num, parent_id):
        self.id_num = id_num
        self.parent_id = parent_id
        self.text = ''
        self.subsections = []
        
    def is_subsection_related(self, text):
        for subsection in self.subsections:
            if subsection.text == text:
                return subsection.id_num
        return('This Section does not contain that subsection.')

    @property
    def clause_type(self):
        pass
    
    @property
    def author(self):
        pass
    
    @property
    def favors(self):
        pass
    
class Subsection(Section):
    
    def __init__(self,id_num, parent_id):
        self.id_num = id_num
        self.parent_id = parent_id
        self.text = ''
        self.clean = ''
    
    @property
    def clause_type(self):
        pass
    
    @property
    def author(self):
        pass
    
    @property
    def favors(self):
        pass
        
        
find_precedent_returned_values = {}
html_template = """
<!DOCTYPE html>
<html>
<head>
<style>
table {
    font-family: arial, sans-serif;
    border-collapse: collapse;
    width: 100%;
}

td, th {
    border: 1px solid #dddddd;
    text-align: left;
    padding: 8px;
}

tr:nth-child(even) {
    background-color: #dddddd;
}
</style>
</head>
<body>

<h2>Comparison Table</h2>

<table>
  <tr>
    <th>Current/Strength</th>
    <th>Alternative</th>
    <th>Comparison</th>
  </tr>
"""
#test
def generate_html():
    html = [html_template]
    df_iter = pd.read_excel('test.xlsx')
    new_clause = ''
    for index, row in df_iter.iterrows():
        if new_clause != str(row['query_raw']):
            html_row = "<tr><th>"+str(row['query_raw'])+"</th><th>"+'----'+"</th><th>" + '----' + "</th></tr>"
        else:
            html_row = "<tr><th>"+str(row['strength'])+"</th><th>"+str(row['match_raw'])+"</th><th>" + str(row['compare']) + "</th></tr>"
        html.append(html_row)
        new_clause = str(row['query_raw'])
    html.append('</table></body></html>')
    html = ''.join(html)
    return html


def segment(text):
    section_list = []
    list_of_section_texts = []
    list_of_section_texts = re.split('[\.|;]\s+[0-9]+\.[0-9]+\s+', text) 
    for i in range(len(list_of_section_texts)):
        new_section = Section(i, i)
        new_section.text = list_of_section_texts[i]
        list_of_sub_section_texts = re.split('[\.|;|:]\s+\([a-zA-Z]\)', new_section.text)
        list_of_defined_terms = re.split('(\.\s*\“\w+\”?)', new_section.text )
        l = len(list_of_defined_terms)
        y=1
        consolidated_defined_terms = []
        for i in range(l):
            if i == 0:
                consolidated_term = list_of_defined_terms[0]
                consolidated_defined_terms.append(consolidated_term)
                join = True
            else:
                if join == True:
                    consolidated_term = list_of_defined_terms[i] + list_of_defined_terms[i+1]
                    consolidated_defined_terms.append(consolidated_term)
                    join = False
                else:
                    join = True
        v=0
        list_of_sub_section_texts =  list_of_sub_section_texts + consolidated_defined_terms
        for sub_section in list_of_sub_section_texts:
            new_subsection = Subsection(i, new_section.id_num)
            raw_clause = re.sub('\\n', "", sub_section)
            raw_clause = re.sub('\n', "", raw_clause)
            raw_clause = re.sub('\t', "", raw_clause)
            raw_clause = re.sub('[\S*|\s*]\\n\S*', "", raw_clause)
            raw_clause = re.sub('\S*htmExhibit\s+[0-9]+\.[0-9]+', "", raw_clause)
            clean_clause = normalize_sent(raw_clause)
            new_subsection.text = raw_clause
            new_subsection.clean = clean_clause
            new_section.subsections.append(new_subsection)
            section_list.append(new_section)
    return section_list

@precedent_button.on_click
def precedent_on_click(b):
    clear_output()
    if precedent_radio.value == 'Load from file':
        to_be_reviewed_text = text_from_file(find_precedent_file_path_text.value)
    else:
        if find_precedent_subject_matter_selection.value == 'Real Estate':
            if find_precedent_text_area.value == '':
                find_precedent_text_area.value = RE_sample
        if find_precedent_subject_matter_selection.value == 'Finance':
            if find_precedent_text_area.value == '':
                find_precedent_text_area.value = finance_sample
        to_be_reviewed_text = find_precedent_text_area.value
    list_of_queries = sent_tokenize(to_be_reviewed_text)
    doc_1_raw_sent_list = []
    doc_1_clean_sent_list = []
    for sent in list_of_queries:
        clean_sent = normalize_sent(sent)
        doc_1_clean_sent_list.append(clean_sent)
        doc_1_raw_sent_list.append(sent)
    df_doc_1 = pd.DataFrame(columns=['raw', 'clean'])
    df_doc_1['clean'] = doc_1_clean_sent_list
    df_doc_1['raw'] = doc_1_raw_sent_list
    if find_precedent_subject_matter_selection.value == 'Finance':
        df_precedent = pd.read_excel('https://s3.amazonaws.com/blaze4/df.xlsx')
        #df_precedent = pd.read_excel('/Users/josiasdewey/jupyter/notebooks/df.xlsx')
    elif find_precedent_subject_matter_selection.value == 'Real Estate':
        #df_precedent = pd.read_excel('/Users/josiasdewey/jupyter/CAN/notebooks/equity_train.xlsx')
        df_precedent = pd.read_excel('https://s3.amazonaws.com/blaze4/equity_train.xlsx')
    df_results = pd.DataFrame(columns=['strength', 'query', 'query_raw', 'match_raw', 'compare'])
    query = df_doc_1['clean']
    choices = df_precedent['Subsection_clean']
    query_list = []
    query_raw_list = []
    found_list = []
    index_list = []
    found_raw_list = []
    strength_list = []
    x = 0
    list_of_raw_query_lists = []
    for q in query:
        match = process.extract(q, choices=choices, scorer=fuzz.token_sort_ratio, limit=int(find_precedent_number_returned.value))
        for tup in match:
            if tup != None:
                index = tup[2]
                if tup[1] >= precedent_compare_slider.value:
                    found = tup[0]
                    found_raw = df_precedent['Subsection'][index]
                    strength = tup[1]
                    index_list.append(index)
                    query_list.append(q)
                    query_raw_list.append(df_doc_1['raw'][x])
                    found_list.append(df_precedent['Subsection_clean'][index])
                    found_raw_list.append(found_raw)
                    strength_list.append(strength)
                list_of_raw_query_lists.append(query_raw_list)
        x = x+1
    compared_file = []
    clean_clause = []
    dmp = diff_match_patch()
    dmp.Diff_Timeout = 0
    for i in range(len(found_raw_list)):
        diff = dmp.diff_main(query_raw_list[i], found_raw_list[i])
        dmp.diff_cleanupSemantic(diff)
        dmp.diff_cleanupSemanticLossless(diff)
        dmp.diff_cleanupEfficiency(diff)
        view = dmp.diff_prettyHtml(diff) 
        compared_file.append(view)
    df_results['compare'] = compared_file
    df_results['query'] = query_list
    df_results['strength'] = strength_list
    df_results['match_raw'] = found_raw_list
    df_results['query_raw'] = query_raw_list
    df_results['index'] = index_list
    find_precedent_returned_values['df_results'] = df_results
    df_results.to_excel('test.xlsx')
    html = generate_html()
    with open('index.html', 'w+') as f:
        f.write(html)
    with precedent_out:
        display(HTML(html))
        #display(df_results)


find_precedent_number_returned = widgets.Dropdown(
    options=['1', '2', '3', '4', '5', '6','7', '8', '9', '10', '25', '50', '100'],
    value='10',
    description='Return:',
    disabled=False,
)

precedent_compare_slider = widgets.FloatSlider(
    value=0,
    base=100,
    min=0, # max exponent of base
    max=100, # min exponent of base
    step=0.2, # exponent step
    description='Similarity:'
)
precedent_compare_slider.layout = {'height' : '100px', 'width' : '400px', 'positioning': 'left'}


find_precedent_number_returned.layout = {'padding' : '15px'} #{'height' : '25px', 'width' : '150px', 'positioning': 'left'}

    
find_precedent_subject_matter_selection = widgets.RadioButtons(
    options=['Finance', 'Litigation', 'Real Estate','Leasing', 'Private Equity', 'Corporate M&A', 'Bond Financing'],
     value='Finance',
    description='Practice:',
    disabled=False
)

precedent_radio = widgets.RadioButtons(
    options=['Load from file', 'Load from text area'],
    value='Load from text area',
    description='Source text:',
    disabled=False
)

#find_precedent_subject_matter_selection.layout = {'height' : '25px', 'width' : '300px', 'positioning': 'left'}
find_precedent_subject_matter_selection.layout = {'padding' : '15px'} #, 'width' : '300px', 'positioning': 'left'}

RE_sample = '''
Seller has good and marketable title to the Property. There are no outstanding rights of first refusal, rights of
reverter or options to purchase relating to the Property or any interest therein. To Sellers knowledge, there are no unrecorded or undisclosed documents or other matters which affect title to the Property. Subject to the Leases, Seller has
enjoyed the continuous and uninterrupted quiet possession, use and operation of the Property, without material complaint or objection by any person. 
'''

finance_sample = '''
The Company will, and will cause each Subsidiary to, maintain insurance with insurers recognized as financially sound and reputable by prudent 
business persons in such forms and amounts and against such risks as the Company reasonably believes is 
prudent and normal within the industry. The Company shall, at the Agent’s request, provide copies to the Agent 
of all insurance policies and other materials related thereto maintained by the Company and its Subsidiaries. 
The Company shall furnish each Bank as soon as available, and in any event no later than each Anniversary Date, 
a summary of its insurance coverage which summary shall be reasonably satisfactory in form and substance to the Banks.
'''

find_precedent_text_area = widgets.Textarea(
    value='',
    description='Clause Text:',
    disabled=False
)

find_precedent_text_area.layout = {'height' : '275px', 'width' : '275px'}

find_precedent_file_path_text = widgets.Text(              
    value='reps.txt',
    placeholder='Type something',
    description='Filename:',
    disabled=False
)

in_progress = widgets.IntProgress(
    value=0,
    min=0,
    max=10,
    step=1,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)


@precedent_download_button.on_click
def find_precedent_download_button_clicked(b):
    output = find_precedent_returned_values['df_results']
    writer = pd.ExcelWriter('precedent_results.xlsx')
    output.to_excel(writer,'Sheet1')
    writer.save()
    precedent_download_button.description = 'Download complete'

def clear_output():
    precedent_out.clear_output()

@precedent_refresh_button.on_click
def find_precedent_refresh_button_clicked(b):
    precedent_out.clear_output()

VBox_thing = VBox(children=[find_precedent_subject_matter_selection,find_precedent_number_returned, precedent_compare_slider])
tab1 = VBox(children=[HBox(children=[find_precedent_text_area, VBox_thing])])
tab2 = VBox(children=[HBox(children=[find_precedent_file_path_text, VBox_thing, precedent_radio])])
tab = widgets.Tab(children=[tab1, tab2])
tab.set_title(0, 'Search by Clause')
tab.set_title(1, 'Search by Document/File')
HBox_precedent = HBox(children=[precedent_button, precedent_refresh_button, precedent_download_button])
VBox_precedent = VBox(children=[tab, HBox_precedent])
VBox_precedent.layout={'border': '.25px solid grey'}

with precedent_UI_out:
    display(VBox_precedent)

display(precedent_UI_out)
precedent_out


Output()

Output(layout=Layout(border='.25px solid grey'))

## QuickCAN *(Coming Soon)*
Computer generated revisions to contracts based on precedent clause database.  Machine learning algorithms and fuzzy matching are used to incorporate revisions made to similar text in precedent clauses.

## QuickCAR *(Coming Soon)*

Computer generated responses to requests for production and interrogetories. Machine learning algorithms and fuzzy matching are used to incorporate historical responses to similar requests.  Will automatically generate a first draft of a response.

## Signature Page Generator
***
Generate signature page packets with the press of a button.

## Closing Binder Generator
***
Automate the preparation of closing binders.

In [18]:
binder_UI_out = widgets.Output(layout={'border': '.25px solid grey'})
binder_out = widgets.Output()
binder_button = create_submit_button()
binder_download_button = create_download_button()
binder_refresh_button = create_refresh_button()

@binder_button.on_click
def on_click(b):
    pass

binder_lender = widgets.RadioButtons(
    options=['Wells Fargo', 'Bank of America (Private Wealth)', 'Bank of America', 'JPMorgan Chase', 'Ocean Bank', 'Other'],
    value='Ocean Bank',
    description='Lender:',
    disabled=False
)

binder_type = widgets.RadioButtons(
    options=['Middle Market', 'Construction Loan', 'Bank Loan', 'Loan Modification', 'ABL Loan', 'Art Loan'],
    value='Bank Loan',
    description='Loan Type:',
    disabled=False
)


tab1 = VBox(children=[HBox(children=[binder_lender, binder_type])])
tab2 = VBox(children=[HBox(children=[])])
tab = widgets.Tab(children=[tab1, tab2])
tab.set_title(0, 'Loan Details')
tab.set_title(1, 'Custom Templates')
HBox_binder = HBox(children=[binder_button, binder_refresh_button, binder_download_button])
VBox_binder = VBox(children=[tab, HBox_binder ])
VBox_binder.layout={'border': '.25px solid grey'}
with binder_UI_out:
    display(VBox_binder)
    
display(binder_UI_out)

Output(layout=Layout(border='.25px solid grey'))